In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split
import os

x_train=np.load("/mnt/sdb/yyamaguchi/simulationB4/bubble_glass_rand/dataset/x_train_five_env_std.npy")
t_train=np.load("/mnt/sdb/yyamaguchi/simulationB4/bubble_glass_rand/dataset/t_train_five_env_std.npy")
bdir="/mnt/sdb/yyamaguchi/simulationB4/bubble_glass_rand/dataset"
odir=os.path.join(bdir,"outputs_env_std")
wdir=os.path.join(odir,"weights")
ldir=os.path.join(odir,"logs")
device="cuda:0"
B=10
W=2500
lr=0.001
N=100
C=5
l1=1e-7
l2=1e-7

class SimpleCNN(nn.Module):
    def __init__(self, input_length, input_channel):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv1d(input_channel, 16, kernel_size=201, padding=100)
        self.bn1 = nn.BatchNorm1d(16)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv1d(16, 4, kernel_size=201, padding=100)
        self.bn2 = nn.BatchNorm1d(4)
        self.relu2 = nn.ReLU()
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(4, 2)
        self.dropout = nn.Dropout(0.1)
        self.ln1 = nn.LayerNorm([16, input_length])
        self.ln2 = nn.LayerNorm([4, input_length+2])
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        # x = self.ln1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        # x = self.ln2(x)
        x = self.pool(x)
        x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x.squeeze(1)

class GradCAM1d:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        self.hook_handles = []
        self._register_hooks()

    def _register_hooks(self):
        def forward_hook(module, input, output):
            self.activations = output.detach()
        def backward_hook(module, grad_in, grad_out):
            self.gradients = grad_out[0].detach()
        self.hook_handles.append(self.target_layer.register_forward_hook(forward_hook))
        self.hook_handles.append(self.target_layer.register_backward_hook(backward_hook))

    def __call__(self, input_tensor):
        self.model.eval()
        input_tensor = input_tensor.to(next(self.model.parameters()).device)
        self.model.zero_grad()
        out = self.model(input_tensor)
        print(out)
        if isinstance(out, (tuple, list)):
            out = out[0]
        target = out.squeeze()
        print(target)
        if target.ndim > 0:
            target = target.sum()
        self.model.zero_grad()
        target.backward(retain_graph=True)
        gradients = self.gradients         # [B, C, L]
        print(gradients.shape)
        activations = self.activations     # [B, C, L]
        print(activations.shape)
        weights = gradients.mean(dim=2, keepdim=True)  # [B, C, 1]
        grad_cam_map = (weights * activations).sum(dim=1, keepdim=True)  # (B,1,L)
        grad_cam_map = torch.relu(grad_cam_map)
        grad_cam_map = torch.nn.functional.interpolate(
            grad_cam_map, size=input_tensor.shape[2], mode='linear', align_corners=False
        )
        grad_cam_map = grad_cam_map.squeeze().cpu().numpy()
        grad_cam_map = (grad_cam_map - grad_cam_map.min()) / (grad_cam_map.max() - grad_cam_map.min() + 1e-8)
        return grad_cam_map

    def remove_hooks(self):
        for handle in self.hook_handles:
            handle.remove()

print(x_train.shape)
# x_train=x_train.reshape(x_train.shape[0],-1)
# x_train=x_train[:,[1,3,4],:]
print(x_train.shape)
x=torch.from_numpy(x_train).float()
t=torch.from_numpy(t_train).float()
# x=x.unsqueeze(1)
print(x.shape)

res_list=[]
coefs_list=[]
reg_list=[]
coefg_list=[]
xgradcam=x[50,:,:]
xgradcam=xgradcam.unsqueeze(0)
dataset=TensorDataset(x,t)
Nttl=len(dataset)
trasize=int(0.7*Nttl)
valsize=int(0.15*Nttl)
testsize=Nttl-trasize-valsize
trainsetall,testset=random_split(dataset,[trasize+valsize,testsize])
testloader=DataLoader(testset,batch_size=1,shuffle=False)

for k in range(10):
    trainset,valset=random_split(trainsetall,[trasize,valsize])
    trainloader=DataLoader(trainset,batch_size=B,shuffle=True,drop_last=True)
    valloader=DataLoader(valset,batch_size=B,shuffle=False)
    model=SimpleCNN(W,C).to(device)

    criterion=nn.MSELoss()
    optimizer=optim.Adam(params=model.parameters(),lr=lr)
    tlosshistory=[]
    vlosshistory=[]

    for epoch in range(N):
        model.train()
        rloss=0.0
        for bx, by in trainloader:
            bx=bx.to(device)
            by=by.to(device)
            optimizer.zero_grad()
            o=model(bx)
            loss=criterion(o,by)
            l1n=sum(p.abs().sum() for p in model.parameters())
            l2n=sum(p.pow(2).sum() for p in model.parameters())
            loss=loss+l1*l1n+l2*l2n
            loss.backward()
            optimizer.step()
            rloss=rloss+loss.item()*bx.size(0)
        eloss=rloss/len(trainloader.dataset)
        tlosshistory.append(eloss)

        model.eval()
        vrloss=0.0
        with torch.no_grad():
            for vx,vy in valloader:
                vx=vx.to(device)
                vy=vy.to(device)
                vo=model(vx)
                vloss=criterion(vo,vy)
                vrloss+=vloss.item()*vx.size(0)
        veloss=vrloss/len(valloader.dataset)
        vlosshistory.append(veloss)

    os.makedirs(odir,exist_ok=True)
    os.makedirs(wdir,exist_ok=True)
    os.makedirs(ldir,exist_ok=True)
    torch.save(model.state_dict(), os.path.join(wdir,f'model{k}.pth'))
    plt.figure()
    plt.plot(range(1, N + 1), [np.log(l) for l in tlosshistory], label='Train Log(Loss)')
    plt.plot(range(1, N + 1), [np.log(l) for l in vlosshistory], label='Validation Log(Loss)')
    plt.title('Learning Curve (Log Loss)')
    plt.xlabel('Epoch')
    plt.ylabel('Log(Loss)')
    plt.rcParams["font.size"] = 16
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(ldir, f'learning_curve_log{k}.png'))
    plt.close()

    plt.figure()
    plt.plot(range(1, N + 1), tlosshistory, label='Train Loss')
    plt.plot(range(1, N + 1), vlosshistory, label='Validation Loss')
    plt.title('Learning Curve (Loss)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.rcParams["font.size"] = 20
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(ldir, f'learning_curve{k}.png'))
    plt.close()

    model.load_state_dict(torch.load(os.path.join(wdir,f'model{k}.pth')))
    model=model.to(device)
    model.eval()
    pred=[]
    targ=[]
    with torch.no_grad():
        for tx,ty in testloader:
            tx=tx.to(device)
            ty=ty.to(device)
            o=model(tx)
            pred.append(o.cpu())
            targ.append(ty.cpu())
    pred=torch.cat(pred,dim=0)
    targ=torch.cat(targ,dim=0)

    pred=pred.numpy()
    targ=targ.numpy()

    print(pred.shape)

    corr=np.corrcoef(targ[:,0],pred[:,0])
    coef=corr[0,1]
    print(f'coefficient solid: {coef}')
    re=np.mean(np.abs(targ[:,0]-pred[:,0])/(targ[:,0]+1e-7))
    print(f're solid: {re}')
    res_list.append(re)
    coefs_list.append(coef)
    corr=np.corrcoef(targ[:,1],pred[:,1])
    coef=corr[0,1]
    print(f'coefficient void: {coef}')
    re=np.mean(np.abs(targ[:,1]-pred[:,1])/(targ[:,1]+1e-7))
    print(f're void: {re}')
    reg_list.append(re)
    coefg_list.append(coef)

    plt.figure(figsize=(8, 8))
    plt.scatter(targ[:,0], pred[:,0], alpha=0.6, marker='o', label='Solid fraction')
    plt.plot([-1, 1], [-1, 1], 'r--', label='Ideal (y=x)')
    plt.title('Predicted vs Actual Values')
    plt.xlabel('Actual Value')
    plt.ylabel('Predicted Value')
    plt.xlim(-0.0, 0.05)
    plt.ylim(-0.0, 0.05)
    plt.rcParams["font.size"] = 20
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(ldir, f'val_pred_vs_actual_solid{k}.png'))
    plt.close()

    plt.figure(figsize=(8, 8))
    plt.scatter(targ[:,1], pred[:,1], alpha=0.6, marker='o', label='Void fraction')
    plt.plot([0, 1], [0, 1], 'r--', label='Ideal (y=x)')
    plt.title('Predicted vs Actual Values')
    plt.xlabel('Actual Value')
    plt.ylabel('Predicted Value')
    plt.xlim(0, 0.4)
    plt.ylim(0, 0.4)
    plt.rcParams["font.size"] = 20
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(ldir, f'val_pred_vs_actual_void{k}.png'))
    plt.close()

    target_layer = None
    if hasattr(model, 'conv3'):
        target_layer = model.conv3
    elif hasattr(model, 'layer3'):
        target_layer = model.layer3
    else:
        for m in reversed(list(model.modules())):
            if isinstance(m, torch.nn.Conv1d):
                target_layer = m
                break
    if target_layer is None:
        raise RuntimeError("Could not find a suitable layer for Grad-CAM.")

    gradcam_output_dir = os.path.join(odir, "gradcam")
    gradcam = GradCAM1d(model, target_layer)
    grad_cam_map = gradcam(xgradcam)
    gradcam.remove_hooks()
    grad_cam_map *= 60
    grad_cam_map -= amp

    plt.figure(figsize=(10,6))
    plt.rcParams['font.size'] = 20
    amp=10
    plt.plot(ta, xgradcamplt[0,:], label='TDX1')
    plt.plot(ta, amp+xgradcamplt[1,:], label='TDX2')
    plt.plot(ta, 2*amp+xgradcamplt[2,:], label='TDX3')
    plt.plot(ta, 3*amp+xgradcamplt[3,:], label='TDX4')
    plt.plot(ta, 4*amp+xgradcamplt[4,:], label='TDX5')
    plt.legend(loc='upper right')
    plt.plot(ta,grad_cam_map[:], color='red', label='Saliency-Map')
    plt.fill_between(ta, grad_cam_map[:],-amp,
            color='red',alpha=0.1)
    plt.xlabel('Time Axis')
    plt.ylabel('Pressure [-]')
    plt.ylim(-amp,5*amp)
    plt.yticks([-amp,0,amp,2*amp,3*amp,4*amp,5*amp],[f"{-amp}","0","0","0","0","0",f"{amp}"])
    plt.xlim(0,W)
    plt.grid(True)
    # plt.xlim(35,40)
    plt.tight_layout()
    new_save_path = os.path.join(odir, f"gradcam{k}.png")
    plt.savefig(new_save_path)
    plt.close()

    model.eval()
    model.zero_grad()

    # 1. まずデバイスに送る
    xgradcam = xgradcam.to(device)

    # 2. デバイス移動「後」に勾配追跡をONにする
    xgradcam.requires_grad_()

    # 3. 予測
    output = model(xgradcam)

    # 4. 逆伝播（クラス1に注目する場合）
    loss = output[0, 1]
    loss.backward()

    # 5. 勾配を取り出す（.gradプロパティに格納されている）
    if xgradcam.grad is not None:
        grads = xgradcam.grad.data # ここで [1, 5, 2500] が取得できる
        
        # --- チャネル別重要度スコアの計算 ---
        # abs()をとって時間方向に平均
        salimap = grads.abs().squeeze(0)
        
        # 0~1に正規化（安全のために 1e-8 を足す）
        salimap = salimap / (salimap.max() + 1e-8)
        
        salimap = salimap.cpu().numpy()
        salimap = salimap*10
        print(salimap.shape)
    else:
        print("勾配が取得できませんでした。requires_gradの位置を確認してください。")

    ta=np.arange(W)

    plt.figure(figsize=(10,6))
    plt.rcParams['font.size'] = 20
    amp=10
    xgradcam=xgradcam.detach().cpu()
    xgradcamplt=xgradcam.numpy().squeeze(0)
    plt.plot(ta, xgradcamplt[0,:], label='TDX1')
    plt.plot(ta, amp+xgradcamplt[1,:], label='TDX2')
    plt.plot(ta, 2*amp+xgradcamplt[2,:], label='TDX3')
    plt.plot(ta, 3*amp+xgradcamplt[3,:], label='TDX4')
    plt.plot(ta, 4*amp+xgradcamplt[4,:], label='TDX5')
    plt.legend(loc='upper right')
    plt.plot(ta,salimap[0,:], color='red', label='Saliency-Map')
    plt.fill_between(ta, salimap[0,:],0,
            color='red',alpha=0.1)
    plt.plot(ta,amp+salimap[1,:], color='red', label='Saliency-Map')
    plt.fill_between(ta, amp+salimap[1,:],amp,
            color='red',alpha=0.1)
    plt.plot(ta,2*amp+salimap[2,:], color='red', label='Saliency-Map')
    plt.fill_between(ta, 2*amp+salimap[2,:],2*amp,
            color='red',alpha=0.1)
    plt.plot(ta,3*amp+salimap[3,:], color='red', label='Saliency-Map')
    plt.fill_between(ta, 3*amp+salimap[3,:],3*amp,
            color='red',alpha=0.1)
    plt.plot(ta,4*amp+salimap[4,:], color='red', label='Saliency-Map')
    plt.fill_between(ta, 4*amp+salimap[4,:],4*amp,
            color='red',alpha=0.1)
    plt.xlabel('Time Axis')
    plt.ylabel('Pressure [-]')
    plt.ylim(-amp,5*amp)
    plt.yticks([-amp,0,amp,2*amp,3*amp,4*amp,5*amp],[f"{-amp}","0","0","0","0","0",f"{amp}"])
    plt.xlim(0,W)
    plt.grid(True)
    # plt.xlim(35,40)
    plt.tight_layout()
    new_save_path = os.path.join(odir, f"saliency{k}.png")
    plt.savefig(new_save_path)
    plt.close()

res_list=np.array(res_list)
coefs_list=np.array(coefs_list)
reg_list=np.array(reg_list)
coefg_list=np.array(coefg_list)
print(f"res_list:{res_list}")
print(f"coefs_list:{coefs_list}")
print(f"reg_list:{reg_list}")
print(f"coefg_list:{coefg_list}")
print(f"res:{np.mean(res_list)}")
print(f"coefs:{np.mean(coefs_list)}")
print(f"reg:{np.mean(reg_list)}")
print(f"coefg:{np.mean(coefg_list)}")

(108, 5, 2500)
(108, 5, 2500)
torch.Size([108, 5, 2500])


/tmp/ipykernel_309389/1875637137.py:196: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(os.path.join(wdir,f'model{k}.pth')))


(17, 2)
coefficient solid: 0.3898019005373705
re solid: 0.5209147930145264
coefficient void: 0.6817193824724055
re void: 0.29079747200012207


/home/user01/Document/yyamaguchi/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1640: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


tensor([[0.0348, 0.1760]], device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor([0.0348, 0.1760], device='cuda:0', grad_fn=<SqueezeBackward0>)
torch.Size([1, 4, 2500])
torch.Size([1, 4, 2500])
(5, 2500)


...

tensor([[0.0339, 0.2810]], device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor([0.0339, 0.2810], device='cuda:0', grad_fn=<SqueezeBackward0>)
torch.Size([1, 4, 2500])
torch.Size([1, 4, 2500])
(5, 2500)


/tmp/ipykernel_309389/1875637137.py:196: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(os.path.join(wdir,f'model{k}.pth')))


(17, 2)
coefficient solid: -0.28130884195746597
re solid: 0.7475320100784302
coefficient void: 0.8132099468736793
re void: 0.30799031257629395


/home/user01/Document/yyamaguchi/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1640: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


tensor([[0.0035, 0.2775]], device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor([0.0035, 0.2775], device='cuda:0', grad_fn=<SqueezeBackward0>)
torch.Size([1, 4, 2500])
torch.Size([1, 4, 2500])
(5, 2500)
res_list:[0.5209148  0.8101612  0.8696227  0.56607467 1.5686723  0.72718656
 0.49155167 0.79074407 0.5975217  0.747532  ]
coefs_list:[ 0.3898019  -0.16474851 -0.01563145  0.22132512  0.23885456  0.19696909
  0.15687695 -0.38038314  0.25270255 -0.28130884]
reg_list:[0.29079747 0.30381244 0.25357535 0.31134292 0.28686866 0.22687106
 0.23903364 0.2477295  0.27793342 0.3079903 ]
coefg_list:[0.68171938 0.73386359 0.7720275  0.71292427 0.65604215 0.7438527
 0.71835957 0.68932733 0.82994945 0.81320995]
res:0.7689981460571289
coefs:0.06144582156434956
reg:0.2745954692363739
coefg:0.7351275900144595


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split
import os

x_train=np.load("/mnt/sdb/yyamaguchi/simulationB4/bubble_glass_rand/dataset/x_train_five_env_std.npy")
t_train=np.load("/mnt/sdb/yyamaguchi/simulationB4/bubble_glass_rand/dataset/t_train_five_env_std.npy")
bdir="/mnt/sdb/yyamaguchi/simulationB4/bubble_glass_rand/dataset"
odir=os.path.join(bdir,"outputs_env_std")
wdir=os.path.join(odir,"weights")
ldir=os.path.join(odir,"logs")

